## Self Supervised Learning (SSL)

In [8]:
!pip install torchgeo --quiet
!pip install lightning --quiet
!pip install prettytable

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 688.1/688.1 kB 15.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.1/246.1 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.3/859.3 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.6/853.6 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.5/760.5 kB 48.1 MB/s eta 0:00:00


In [2]:
import rasterio

# Path to your image
img_path = "/kaggle/input/datasets/apollo2506/eurosat-dataset/EuroSATallBands/AnnualCrop/AnnualCrop_1.tif"

# Open the TIFF
with rasterio.open(img_path) as src:
    print(f"Number of bands: {src.count}")
    print(f"Width: {src.width}, Height: {src.height}")
    
    # Iterate over bands
    for i in range(1, src.count + 1):  # rasterio bands are 1-indexed
        band = src.read(i)
        print(f"Band {i} shape: {band.shape}, dtype: {band.dtype}")



Number of bands: 13
Width: 64, Height: 64
Band 1 shape: (64, 64), dtype: uint16
Band 2 shape: (64, 64), dtype: uint16
Band 3 shape: (64, 64), dtype: uint16
Band 4 shape: (64, 64), dtype: uint16
Band 5 shape: (64, 64), dtype: uint16
Band 6 shape: (64, 64), dtype: uint16
Band 7 shape: (64, 64), dtype: uint16
Band 8 shape: (64, 64), dtype: uint16
Band 9 shape: (64, 64), dtype: uint16
Band 10 shape: (64, 64), dtype: uint16
Band 11 shape: (64, 64), dtype: uint16
Band 12 shape: (64, 64), dtype: uint16
Band 13 shape: (64, 64), dtype: uint16


In [11]:
import torch
torch.cuda.empty_cache()

# ========================
# Imports
# ========================
import os
import random
from datetime import datetime
import glob
import argparse

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, Subset
import rasterio
from rasterio.enums import Resampling
from prettytable import PrettyTable
from lightning.pytorch.loggers import CSVLogger
from torchvision import transforms
from lightning.pytorch import Trainer
import kornia.augmentation as K
from dataclasses import dataclass
from torchgeo.trainers.moco import MoCoTask
from torchgeo.models.resnet import ResNet18_Weights
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import pandas as pd
from lightning.pytorch.callbacks import ModelCheckpoint
import time

# ========================
# Reproducibility
# ========================
seed = 42
os.environ["PYTHONHASHSEED"] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
import pytorch_lightning as pl
pl.seed_everything(seed, workers=True)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

# Timestamp for logging / checkpoints
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(seed)

CLASS_NAMES = [
    "AnnualCrop",
    "Forest",
    "HerbaceousVegetation",
    "Highway",
    "Industrial",
    "Pasture",
    "PermanentCrop",
    "Residential",
    "River",
    "SeaLake"
]

CLASS_TO_IDX = {name: idx for idx, name in enumerate(CLASS_NAMES)}

Seed set to 42


In [13]:
@dataclass
class DataConfig:
    data_root_dir: str = "/home/krschap/rabina/data/s2a"
    split_path: str = "eurosat_all_bands_split.csv"
    stats_split: str = "full_ssl"  # options: full_ssl, subset1, subset2, subset3, subset4
    split: str = "subset1"
    compute_stats: bool = True
    # n_samples: int = None
    batch_size: int = 64
    patch_size: int = 264
    num_workers: int = 1

@dataclass
class TrainingConfig:
    experiment_out_dir: str = f"ssl_moco_{timestamp}"
    model: str = "resnet18"
    in_channels: int = 13
    version: int = 2
    lr: float = 1e-4
    use_peft: bool = False
    temperature: float = 0.15
    memory_bank_size: int = 2048
    target_size: int = 224
    max_epochs: int = 100
    batch_size: int =32
    #devices = []

def split_dataset(root_dir,csv_output):
    # --- COLLECT IMAGE INFO ---
    data=[]
    # Iterate over each class folder
    for class_name in os.listdir(root_dir):
        class_path = os.path.join(root_dir, class_name)
        # print(class_path)
        if not os.path.isdir(class_path):
            continue  # skip files in root_dir
    
        # Iterate over images in the class folder
        for fname in os.listdir(class_path):
            # if fname.lower().endswith((".jpg", ".jpeg", ".png")):
            if fname.lower().endswith((".tif", ".tiff")):
                # path = os.path.join(class_path, fname)
                rel_path = os.path.join(class_name, fname)
                data.append({
                    "id": os.path.splitext(fname)[0].split("_")[-1],
                     "fname": fname,
                    "rel_path": rel_path,
                    "label": class_name
                })
    # --- CREATE DATAFRAME ---
    df = pd.DataFrame(data)
    # print(df.columns)
    # --- STRATIFIED SPLIT: 80% SSL, 20% Downstream ---
    ssl_df, downstream_df = train_test_split(
        df,
        test_size=0.2,
        stratify=df['label'],
        random_state=42
    )
    
    ssl_df['task'] = 'ssl'
    downstream_df['task'] = 'downstream'
    
    df = pd.concat([ssl_df, downstream_df]).reset_index(drop=True)
    
    ssl_df = df[df['task'] == 'ssl'].copy()
    
    skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
    
    ssl_splits = np.empty(len(ssl_df), dtype=object)
    split_names = ['subset1', 'subset2', 'subset3', 'subset4']
    
    for fold_idx, (_, val_idx) in enumerate(skf.split(ssl_df, ssl_df['label'])):
        ssl_splits[val_idx] = split_names[fold_idx]
    
    df.loc[ssl_df.index, 'split'] = ssl_splits
    
    down_df = df[df['task'] == 'downstream'].copy()
    # Step 1: Train (70%) vs Temp (30%)
    train_df, temp_df = train_test_split(
        down_df,
        test_size=0.30,
        stratify=down_df['label'],
        random_state=42
    )
    # Step 2: Temp → Val (15%) + Test (15%)
    val_df, test_df = train_test_split(
        temp_df,
        test_size=0.50,  # half of 30% = 15%
        stratify=temp_df['label'],
        random_state=42
    )
    # Assign splits back
    df.loc[train_df.index, 'split'] = 'train'
    df.loc[val_df.index, 'split'] = 'val'
    df.loc[test_df.index, 'split'] = 'test'
    
    # --- FINAL CHECK ---
    print(df['task'].value_counts())
    print(df['split'].value_counts())
    # print(df.head(10))
    
    # --- SAVE CSV ---
    df.to_csv(csv_output, index=False)
    print(f"CSV saved to {csv_output}")

class EurosatDataset(Dataset):
    def __init__(self, data_dir, split_path, split, transforms=None):
        """
        Args:
            data_dir (str): Eurosat folder paths.
            split_path (str): CSV file path containing splits metadata
            split (str): all, full_ssl, subset1, subset2, subset3, subset4, train, test, val
            transforms (callable, optional): Optional transform to apply to patches.
        """
        self.data_dir = data_dir
        self.split_path = split_path
        self.transforms = transforms
        
        # Precompute all paths based on split
        self.samples = []
        self.df= None
        df=pd.read_csv(split_path)
        
        if split=="full_ssl":
            self.df=df[df['task'] == "ssl"].reset_index(drop=True)
        else:
            self.df=df[df['split'] == split].reset_index(drop=True)
        self.samples = self.df['rel_path'].tolist()

        print(f"len(df): {len(self.df)}")
        print(f"len(samples): {len(self.samples)}")
        

    def __len__(self):
        return len(self.samples)
    

    # def __getitem__(self, idx):
    #     sample_path = os.path.join(self.data_dir, self.samples[idx])
        
    #     with Image.open(sample_path) as img:
    #         patch_tensor = img.convert("RGB")
    #     if self.transforms:
    #         patch_tensor = self.transforms(patch_tensor)

    #     return {"image": patch_tensor}
    
    def __getitem__(self, idx):
        sample_path = os.path.join(self.data_dir, self.samples[idx])
    
        # --- Read TIFF with rasterio ---
        with rasterio.open(sample_path) as src:
            # Read all bands as float32
            bands = [src.read(b).astype(np.float32) for b in range(1, src.count + 1)]
        
        # Stack bands to shape [C, H, W]
        img_array = np.stack(bands, axis=0)
    
        # --- Convert to torch tensor ---
        patch_tensor = torch.tensor(img_array, dtype=torch.float32)
    
        # --- Apply transforms if provided ---
        if self.transforms:
            patch_tensor = self.transforms(patch_tensor)

        # --- map string label to int ---
        class_name = self.df.iloc[idx]['label']  # the string label
        label_idx = CLASS_TO_IDX[class_name]
    
        # return patch_tensor, label_idx
    
        return {"image": patch_tensor, "label": label_idx}

def calculate_stats_parallel(dataset, batch_size=16, num_workers=4):

    n_samples = len(dataset)
    print(f"Total samples in dataset: {n_samples}")

    # if n_samples is not None:
    #     n = min(total, n_samples)
    #     print(f"Calculating stats on {n} randomly selected samples...")
    #     # Randomly select a subset of indices for efficiency
    #     np.random.seed(seed)
    #     indices = np.random.choice(total, size=n, replace=False)
    #     subset = Subset(dataset, indices)
    # else:
    print(f"Calculating stats on the entire dataset...")
    # subset = dataset
    # n_samples=total

    loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        worker_init_fn=seed_worker,
        generator=g
    )

    channel_sum = 0.0
    channel_sum_sq = 0.0
    num_pixels = 0

    total_batches = len(loader)
    print(f"Total batches to process: {total_batches}")
    for batch_idx, batch in enumerate(loader, start=1):
        imgs = batch["image"]
        b, c, h, w = imgs.shape
        channel_sum += imgs.sum(dim=(0, 2, 3))
        channel_sum_sq += (imgs**2).sum(dim=(0, 2, 3))
        num_pixels += b * h * w

        if batch_idx % 100 == 0 or batch_idx == total_batches:
            print(f"Processed batch {batch_idx}/{total_batches} ")
                # f"≈ {batch_idx * b}/{n_samples} samples")

    mean = channel_sum / num_pixels
    torch.set_printoptions(sci_mode=False, precision=4)
    # std  = torch.sqrt(channel_sum_sq / num_pixels - mean**2)
    variance = channel_sum_sq / num_pixels - mean**2
    variance = torch.clamp(variance, min=0)  # Avoid negative values
    # Add small epsilon to avoid sqrt(0) issues
    std = torch.sqrt(variance + 1e-8)
    return mean, std

def summary_trainable(model):
    table = PrettyTable()
    table.field_names = ["Module", "Type", "Trainable Params", "Total Params"]

    for name, module in model.named_children():
        total_params = sum(p.numel() for p in module.parameters())
        trainable_params = sum(p.numel() for p in module.parameters() if p.requires_grad)
        table.add_row([name, type(module).__name__, f"{trainable_params:,}", f"{total_params:,}"])

    total_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    
    print(table)
    print(f"Total trainable parameters: {total_trainable:,} ({total_trainable / 1e6:.2f} M)")
    print(f"Total parameters: {total_params:,} ({total_params / 1e6:.2f} M)")

# def main(data_root_dir, n_samples,  batch_size, patch_size, num_workers):
def main(data_cfg, training_cfg):
    print("Data root directory:", data_cfg.data_root_dir)
    print("========================")
    print("Dataset config:", data_cfg)
    print("========================")
    print("Training config:", training_cfg)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    os.makedirs(training_cfg.experiment_out_dir, exist_ok=True)
    logger = CSVLogger(training_cfg.experiment_out_dir, name=f"metrics_{timestamp}")

    aug = K.AugmentationSequential(
        K.RandomResizedCrop(size=(training_cfg.target_size, training_cfg.target_size), scale=(0.4, 1.0)),
        K.RandomHorizontalFlip(),
        K.RandomVerticalFlip(),
        K.RandomGaussianBlur(kernel_size=(7,7), sigma=(0.1, 1.5), p=0.3),
        K.RandomBrightness(brightness=(0.85, 1.15), p=0.5),
        data_keys=['input'],
    )

    if not os.path.exists(data_cfg.data_root_dir):
        raise FileNotFoundError(f"Data root directory does not exist: {data_cfg.data_root_dir}")
    scenes = sorted(glob.glob(os.path.join(data_cfg.data_root_dir, "*/")))
    bands = ["B1","B2","B3","B4","B5","B6","B7","B8","B8A","B9","B11","B12"]
    # ========================
    # Compute dataset statistics (mean, std)
    # ========================
    if data_cfg.compute_stats:
        # start_time = time.time()
        #scenes = sorted(glob.glob(os.path.join(data_cfg.data_root_dir, "*/")))
        # end_time = time.time()

        # print(f"Found {len(scenes)} scenes in {end_time-start_time:.2f} seconds")

        # bands = ["B1","B2","B3","B4","B5","B6","B7","B8","B8A","B9","B11","B12"]
        # temp_dataset = SSLDataset(scenes, bands, patch_size=data_cfg.patch_size)
        
        temp_dataset = EurosatDataset(
            data_dir = data_cfg.data_root_dir, 
            split_path= data_cfg.split_path,
            split = data_cfg.stats_split,
        )
        import time
        start_time = time.time()
        mean, std = calculate_stats_parallel(temp_dataset, batch_size=data_cfg.batch_size, num_workers=data_cfg.num_workers)

        # mean, std = calculate_stats_parallel(temp_dataset, n_samples=data_cfg.n_samples, batch_size=data_cfg.batch_size, num_workers=data_cfg.num_workers)
        end_time = time.time()
        print(f"calculate_stats time: {(end_time-start_time)/60:.2f} min")
        print("Mean:", mean)
        print("Std:", std)
        mean = mean.tolist()
        std = std.tolist()
    else:
        print("Using pre-computed mean and std")
        mean =[1354.4231, 1116.6886, 1035.6735,  938.2608, 1184.0359, 1969.5629,
        2332.6990, 2260.2139,  723.8848,   13.1612, 1785.6721, 1101.3131,
        2549.8162]
        
        std= [ 243.4430,  331.7848,  396.9748,  596.1298,  577.0484,  887.0798,
        1116.7583, 1146.2135,  404.7274,    9.5821, 1028.9261,  766.2137,
        1271.2732]

    # ========================
    # Train MoCo model
    # ========================
    transform = transforms.Compose([
        transforms.Resize((training_cfg.target_size, training_cfg.target_size)),
        transforms.Normalize(mean=mean, std=std)
    ])
    dataset= EurosatDataset(data_cfg.data_root_dir, data_cfg.split_path, data_cfg.split, transforms=transform)
    # dataset = SSLDataset(scenes, bands, transforms=transform, patch_size=training_cfg.target_size)
    print(len(dataset))
    print(dataset[0]['image'].shape)

    data_loader = DataLoader(
        dataset,
        batch_size=training_cfg.batch_size,
        shuffle=False,
        num_workers=data_cfg.num_workers,
        worker_init_fn=seed_worker,
        generator=g
    )
    num_batches = len(data_loader)
    print("Number of batches:", num_batches)

    task = MoCoTask(
        model=training_cfg.model,      
        weights= None,
        # weights= ResNet18_Weights.SENTINEL2_ALL_MOCO,
        in_channels=training_cfg.in_channels,       
        version=training_cfg.version,             # MoCo v2
        size=training_cfg.target_size,          
        augmentation1=aug,
        augmentation2=aug,
        lr=training_cfg.lr,
        memory_bank_size=training_cfg.memory_bank_size,
        temperature=training_cfg.temperature,
    )

    # -----------------------------
    # PEFT / Full Fine-Tuning Logic
    # -----------------------------
    if training_cfg.use_peft:
        print("Using PEFT: freezing backbone except last block, training projection head...")
        for name, param in task.backbone.named_parameters():
            if "layer4" in name:      # optionally fine-tune last residual block
                param.requires_grad = True
            else:
                param.requires_grad = False
    else:
        print("Full fine-tuning: backbone and projection head trainable...")
        for param in task.backbone.parameters():
            param.requires_grad = True

    # Momentum backbone always frozen
    for param in task.backbone_momentum.parameters():
        param.requires_grad = False

    # Projection head always trainable
    for param in task.projection_head.parameters():
        param.requires_grad = True

    # Example usage for your task
    summary_trainable(task)

    checkpoint_callback = ModelCheckpoint(
      dirpath=training_cfg.experiment_out_dir,
      # filename="ssl-best-{epoch:02d}",
      monitor="train_loss",
      mode="min",
      save_top_k=1,
      save_last=True
    )


    trainer = Trainer(
        max_epochs=training_cfg.max_epochs,
        enable_progress_bar=True, 
        log_every_n_steps=num_batches,
        precision=16,
        accelerator="gpu" if torch.cuda.is_available() else "cpu",
        devices=[0] if torch.cuda.is_available() else 1,
    	deterministic=True,
        callbacks=[checkpoint_callback],
        logger=logger)
    import time
    start_time=time.time()
    trainer.fit(task, data_loader)
    end_time=time.time()
    print(f"Training time: {(end_time-start_time)/60} min")

    # torch.save(task.backbone.state_dict(),f"{training_cfg.experiment_out_dir}/ssl_backbone_{timestamp}.pth")
    # torch.save(task.projection_head.state_dict(), f"{training_cfg.experiment_out_dir}/projection_head_{timestamp}.pth")
    # trainer.save_checkpoint(f"{training_cfg.experiment_out_dir}/ssl_ckpt_{timestamp}.ckpt")



In [5]:
split_dataset(root_dir="/kaggle/input/datasets/apollo2506/eurosat-dataset/EuroSATallBands",
              csv_output="/kaggle/working/eurosat_all_bands_split.csv")

task
ssl           22077
downstream     5520
Name: count, dtype: int64
split
subset1    5520
subset4    5519
subset2    5519
subset3    5519
train      3864
test        828
val         828
Name: count, dtype: int64
CSV saved to /kaggle/working/eurosat_all_bands_split.csv


In [6]:
if __name__ == "__main__":
    # device =  "gpu"
    target_num_workers = int(os.cpu_count()*0.75)  # Use 75% of available CPU cores
    # print("Using device:", device)
    print("CPU cores available:", os.cpu_count())
    print("CPU cores using:", target_num_workers)

    parser = argparse.ArgumentParser(description="Calculate dataset statistics")
    parser.add_argument(
        "--data_root_dir",
        type=str,
        default="/kaggle/input/datasets/apollo2506/eurosat-dataset/EuroSATallBands",
        help="Path to the root directory of scenes"
    )
    
    parser.add_argument(
        "--split_path",
        type=str,
        default="/kaggle/working/eurosat_all_bands_split.csv",
        help="Path to the split file"
    )

    # parser.add_argument(
    #     "--n_samples",
    #     type=int,
    #     default=500, # Set to None to use the entire dataset
    #     help="Number of samples to calculate statistics on"
    # )

    parser.add_argument(
        "--num_workers",
        type=int,
        default=target_num_workers,
        help="Number of workers for data loading"
    )
    
    # args = parser.parse_args()
    args, unknown = parser.parse_known_args()

    # Training configuration
    
    
    data_cfg = DataConfig(
        data_root_dir=args.data_root_dir,
        split_path=args.split_path,
        stats_split="full_ssl", # options: full_ssl, subset1, subset2, subset3, subset4
        split ="full_ssl",
        compute_stats=True,
        # n_samples=args.n_samples, # only used for stats calculation, not training
        num_workers=args.num_workers,
        batch_size=64, # only used for stats calculation, not training
        patch_size=64, # only used for stats calculation, not training
    )

    training_cfg = TrainingConfig(
        experiment_out_dir=f"/kaggle/working/output/full_ssl_v1",
        model="resnet18",
        # weights= ResNet50_Weights.SENTINEL2_ALL_MOCO,
        in_channels=13,
        version=2,
        lr=0.015, #lr = 0.03 × (batch_size / 256)
        # weight_decay=1e-4,
        # optimizer="sgd",
        # scheduler="cosine",
        use_peft=False,
        temperature=0.15,
        memory_bank_size=4096, #4096, #2048
        target_size=64,
        batch_size=128,
        max_epochs=100,
    )
    main(data_cfg, training_cfg)


CPU cores available: 4
CPU cores using: 3
Data root directory: /kaggle/input/datasets/apollo2506/eurosat-dataset/EuroSATallBands
Dataset config: DataConfig(data_root_dir='/kaggle/input/datasets/apollo2506/eurosat-dataset/EuroSATallBands', split_path='/kaggle/working/eurosat_all_bands_split.csv', stats_split='full_ssl', split='full_ssl', compute_stats=True, batch_size=64, patch_size=64, num_workers=3)
Training config: TrainingConfig(experiment_out_dir='/kaggle/working/output/full_ssl_v1', model='resnet18', in_channels=13, version=2, lr=0.015, use_peft=False, temperature=0.15, memory_bank_size=4096, target_size=64, max_epochs=100, batch_size=128)
len(df): 22077
len(samples): 22077
Total samples in dataset: 22077
Calculating stats on the entire dataset...
Total batches to process: 345
Processed batch 100/345 
Processed batch 200/345 
Processed batch 300/345 
Processed batch 345/345 
calculate_stats time: 3.84 min
Mean: tensor([1354.4231, 1116.6886, 1035.6735,  938.2608, 1184.0359, 1969.56

/usr/local/lib/python3.12/dist-packages/torchgeo/trainers/moco.py:209: UserWarning: MoCo v2 only uses 2 layers in its projection head
  warnings.warn('MoCo v2 only uses 2 layers in its projection head')
/usr/local/lib/python3.12/dist-packages/lightning/fabric/connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.


Full fine-tuning: backbone and projection head trainable...
+--------------------------+------------------------+------------------+--------------+
|          Module          |          Type          | Trainable Params | Total Params |
+--------------------------+------------------------+------------------+--------------+
|         backbone         |         ResNet         |    11,207,872    |  11,207,872  |
|    backbone_momentum     |         ResNet         |        0         |  11,207,872  |
|     projection_head      |   MoCoProjectionHead   |    19,931,392    |  19,931,392  |
| projection_head_momentum |   MoCoProjectionHead   |        0         |  19,931,392  |
|        criterion         |       NTXentLoss       |        0         |      0       |
|      augmentation1       | AugmentationSequential |        0         |      0       |
+--------------------------+------------------------+------------------+--------------+
Total trainable parameters: 31,139,264 (31.14 M)
Total param

/usr/local/lib/python3.12/dist-packages/lightning/pytorch/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
/usr/local/lib/python3.12/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:881: Checkpoint directory /kaggle/working/output/full_ssl_v1 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/usr/local/lib/python3.12/dist-packages/lightning/pytorch/utilities/model_summary/model_summary.py:242: Precision 16-mixed is not supported by the model summary.  Estimated model size in MB will not be accurate. Using 32 bits instead.


┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name                     ┃ Type                   ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ backbone                 │ ResNet                 │ 11.2 M │ train │     0 │
│ 1 │ backbone_momentum        │ ResNet                 │ 11.2 M │ train │     0 │
│ 2 │ projection_head          │ MoCoProjectionHead     │ 19.9 M │ train │     0 │
│ 3 │ projection_head_momentum │ MoCoProjectionHead     │ 19.9 M │ train │     0 │
│ 4 │ criterion                │ NTXentLoss             │      0 │ train │     0 │
│ 5 │ augmentation1            │ AugmentationSequential │      0 │ train │     0 │
└───┴──────────────────────────┴────────────────────────┴────────┴───────┴───────┘

Trainable params: 31.1 M                                                                                           
Non-trainable params: 31.1 M                                                                                       
Total params: 62.3 M                                                                                               
Total estimated model params size (MB): 249                                                                        
Modules in train mode: 214                                                                                         
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=10000.0 (msgs/sec)
ServerApp.rate_limit_window=1.0 (secs)

`Trainer.fit` stopped: `max_epochs=100` reached.


Training time: 138.5041580716769 min


In [1]:
import shutil

shutil.make_archive(
    "/kaggle/working/output/full_ssl_v1", 
    'zip', 
    "/kaggle/working/output/full_ssl_v1"
)

'/kaggle/working/output/full_ssl_v1.zip'

Model Training

In [14]:
import os
import torch
import torch.nn as nn
import pytorch_lightning as pl
# from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import ModelCheckpoint
# from torch.utils.data import DataLoader
import torchvision.transforms as T
# from torchvision.datasets import ImageFolder
from torchvision.models import resnet18
from pytorch_lightning.callbacks import EarlyStopping
from torchmetrics.classification import MulticlassF1Score, MulticlassAccuracy
from datetime import datetime

from prettytable import PrettyTable
from lightning.pytorch.loggers import CSVLogger
from torchvision import transforms
from lightning.pytorch import Trainer

# ---------- Downstream Lightning Module ----------
class EuroSATClassifier(pl.LightningModule):
    def __init__(self, backbone, num_classes, lr=1e-3):
        super().__init__()
        self.backbone = backbone
        # freeze backbone by default
        for p in self.backbone.parameters():
            p.requires_grad = False

        # classification head
        # flatten features and map to num_classes
        self.classifier = nn.Linear(self.backbone.out_features, num_classes)
        self.criterion = nn.CrossEntropyLoss()
        self.lr = lr
        #train 
        # Metrics
        self.train_acc = MulticlassAccuracy(num_classes=num_classes)
        self.val_acc = MulticlassAccuracy(num_classes=num_classes)
        
        self.train_f1_macro = MulticlassF1Score(num_classes=num_classes, average="macro")
        
        # Validation metrics
        self.val_f1_macro = MulticlassF1Score(num_classes=num_classes, average="macro")
        

    def forward(self, x):
        feats = self.backbone(x)
        feats = feats.flatten(1)
        out = self.classifier(feats)
        return out

    def training_step(self, batch, batch_idx):
        # x, y = batch
        x, y = batch['image'], batch['label']
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
    
        # update metrics
        self.train_acc(y_hat, y)
        self.train_f1_macro(y_hat, y)
    
        # log
        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", self.train_acc, prog_bar=False, on_epoch=True, on_step=False)
        self.log("train_f1_macro", self.train_f1_macro, prog_bar=True, on_epoch=True, on_step=False)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch['image'], batch['label']
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
    
        # update metrics
        self.val_acc(y_hat, y)
        self.val_f1_macro(y_hat, y)
    
        # log
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", self.val_acc, prog_bar=False, on_epoch=True, on_step=False)
        self.log("val_f1_macro", self.val_f1_macro, prog_bar=True, on_epoch=True, on_step=False)
    
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.classifier.parameters(), lr=self.lr)


# --------- Load pretrained backbone from MoCo checkpoint ---------

# def load_moco_backbone(ckpt_path: str):
#     """
#     Load a MoCo pretrained backbone from SSL checkpoint.
#     Assumes the checkpoint stores a ResNet style encoder
#     with feature dimension in `backbone`.
#     """

#     ckpt = torch.load(ckpt_path, map_location="cpu")
#     state_dict = ckpt["state_dict"] if "state_dict" in ckpt else ckpt

#     # assume the backbone key prefix is something like "backbone."
#     # strip off the moco projection head weights
#     backbone_keys = {k.replace("backbone.", ""):v
#                      for k, v in state_dict.items()
#                      if k.startswith("backbone") and "projection_head" not in k}

#     # create a ResNet50 backbone without final FC
#     model = resnet18(weights=None)
#     layers = list(model.children())[:-1]  # all layers except final fc
#     backbone = nn.Sequential(*layers)

#     # load state dict into backbone
#     backbone.load_state_dict(backbone_keys, strict=False)

#     # record output dim (ResNet50 last conv features)
#     backbone.out_features = model.fc.in_features

#     return backbone

from torchvision.models import resnet18

def load_moco_backbone(ckpt_path: str, in_channels=13):
    ckpt = torch.load(ckpt_path, map_location="cpu")
    state_dict = ckpt.get("state_dict", ckpt)

    # Strip MoCo projection head
    backbone_keys = {
        k.replace("backbone.", ""): v
        for k, v in state_dict.items()
        if k.startswith("backbone") and "projection_head" not in k
    }

    # Load ResNet18
    model = resnet18(weights=None)
    
    # Replace first conv layer to accept 13 channels
    model.conv1 = nn.Conv2d(
        in_channels,
        model.conv1.out_channels,
        kernel_size=model.conv1.kernel_size,
        stride=model.conv1.stride,
        padding=model.conv1.padding,
        bias=model.conv1.bias is not None
    )

    # Keep all layers except the final FC
    layers = list(model.children())[:-1]
    backbone = nn.Sequential(*layers)
    backbone.out_features = model.fc.in_features

    # Load MoCo weights into backbone (ignore missing keys for conv1)
    backbone.load_state_dict(backbone_keys, strict=False)

    return backbone


# --------------- Main Function -----------------

def main():
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument("--data_root_dir", type=str, default="/kaggle/input/datasets/apollo2506/eurosat-dataset/EuroSATallBands",
                        help="Path to EuroSAT downstream data with subfolders per class")
    parser.add_argument("--split_path", type=str, default="/kaggle/working/eurosat_all_bands_split.csv",
                        help="Path to split csv")
    parser.add_argument("--ckpt", type=str, default="/kaggle/working/output/full_ssl_v1/epoch=91-step=15916.ckpt",
                        help="Path to your MoCo SSL pretrained checkpoint")
    parser.add_argument("--max_epochs", type=int, default=100)
    parser.add_argument("--experiment_out_dir", type=str, default="/kaggle/working/full_downstream_v1")
    
    num_workers = int(os.cpu_count()*0.75) 
    lr=1e-3
    batch_size=32
    target_size=64
    
    args, _ = parser.parse_known_args()

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    os.makedirs(args.experiment_out_dir, exist_ok=True)

    
    logger = CSVLogger(args.experiment_out_dir, name=f"metrics_{timestamp}")


    # -------- Load Backbone --------
    backbone = load_moco_backbone(args.ckpt, in_channels=13)

    # -------- Build Classifier LightningModule --------
    # EuroSAT has 10 classes
    model = EuroSATClassifier(backbone, num_classes=10, lr=lr)

    print("Using pre-computed mean and std")
    
    mean =[1354.4231, 1116.6886, 1035.6735,  938.2608, 1184.0359, 1969.5629,
        2332.6990, 2260.2139,  723.8848,   13.1612, 1785.6721, 1101.3131,
        2549.8162]
        
    std= [ 243.4430,  331.7848,  396.9748,  596.1298,  577.0484,  887.0798,
    1116.7583, 1146.2135,  404.7274,    9.5821, 1028.9261,  766.2137,
    1271.2732]

    # -------- Data Transforms --------
    transform = transforms.Compose([
        transforms.Resize((target_size, target_size)),
        transforms.Normalize(mean=mean, std=std)
    ])
    
    train_ds= EurosatDataset(args.data_root_dir, args.split_path, "train", transforms=transform)
    val_ds= EurosatDataset(args.data_root_dir, args.split_path, "val", transforms=transform)
    
    train_loader = DataLoader(train_ds, 
                              batch_size=batch_size,
                              shuffle=True, 
                              num_workers=num_workers, 
                              worker_init_fn=seed_worker,
                              generator=g)
    val_loader   = DataLoader(val_ds, 
                              batch_size=batch_size,
                              shuffle=False, 
                              num_workers=num_workers,
                              worker_init_fn=seed_worker,
                              generator=g)

    # -------- Trainer Setup --------
    checkpoint_cb = ModelCheckpoint(
        dirpath=args.experiment_out_dir,
        monitor="val_f1_macro",
        mode="max", 
        save_top_k=1,
        save_last=True 
    )
    early_stop_cb = EarlyStopping(
        monitor="val_f1_macro",
        mode="max",
        patience=20,
        min_delta=0.005,
        verbose=True
    )

    # lr_monitor_cb = LearningRateMonitor(logging_interval="epoch")
    trainer = pl.Trainer(
        accelerator="gpu" if torch.cuda.is_available() else "cpu",
        devices=[0] if torch.cuda.is_available() else 1,
        max_epochs=args.max_epochs,
        callbacks=[checkpoint_cb, early_stop_cb],
        precision=16,
    	deterministic=True,
        log_every_n_steps=len(train_loader),
        enable_progress_bar=True,
        logger=logger
    )

    # -------- Train --------
    trainer.fit(model, train_loader, val_loader)


if __name__ == "__main__":
    main()

/usr/local/lib/python3.12/dist-packages/lightning_fabric/connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
Using 16bit Automatic Mixed Precision (AMP)


Using pre-computed mean and std
len(df): 3864
len(samples): 3864
len(df): 828
len(samples): 828


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
/usr/local/lib/python3.12/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:881: Checkpoint directory /kaggle/working/full_downstream_v1 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/usr/local/lib/python3.12/dist-packages/pytorch_lightning/utilities/model_summary/model_summary.py:242: Precision 16-mixed is not supported by the model summary.  Estimated model size in MB will not be accurate. Using 32 bits instead.


┏━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name           ┃ Type               ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ backbone       │ Sequential         │ 11.2 M │ train │     0 │
│ 1 │ classifier     │ Linear             │  5.1 K │ train │     0 │
│ 2 │ criterion      │ CrossEntropyLoss   │      0 │ train │     0 │
│ 3 │ train_acc      │ MulticlassAccuracy │      0 │ train │     0 │
│ 4 │ val_acc        │ MulticlassAccuracy │      0 │ train │     0 │
│ 5 │ train_f1_macro │ MulticlassF1Score  │      0 │ train │     0 │
│ 6 │ val_f1_macro   │ MulticlassF1Score  │      0 │ train │     0 │
└───┴────────────────┴────────────────────┴────────┴───────┴───────┘

Trainable params: 5.1 K                                                                                            
Non-trainable params: 11.2 M                                                                                       
Total params: 11.2 M                                                                                               
Total estimated model params size (MB): 44                                                                         
Modules in train mode: 73                                                                                          
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

Metric val_f1_macro improved. New best score: 0.427
Metric val_f1_macro improved by 0.067 >= min_delta = 0.005. New best score: 0.494
Metric val_f1_macro improved by 0.049 >= min_delta = 0.005. New best score: 0.542
Metric val_f1_macro improved by 0.059 >= min_delta = 0.005. New best score: 0.601
Metric val_f1_macro improved by 0.013 >= min_delta = 0.005. New best score: 0.614
Metric val_f1_macro improved by 0.022 >= min_delta = 0.005. New best score: 0.636
Metric val_f1_macro improved by 0.006 >= min_delta = 0.005. New best score: 0.642
Metric val_f1_macro improved by 0.008 >= min_delta = 0.005. New best score: 0.650
Metric val_f1_macro improved by 0.012 >= min_delta = 0.005. New best score: 0.662
Monitored metric val_f1_macro did not improve in the last 20 records. Best score: 0.662. Signaling Trainer to stop.


In [15]:
import shutil

shutil.make_archive(
    "/kaggle/working/full_downstream_v1", 
    'zip', 
    "/kaggle/working/full_downstream_v1"
)

'/kaggle/working/full_downstream_v1.zip'